# **Libraries**





In [ ]:
# Data manipulation
# ==============================================================================
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Plots
# ==============================================================================
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
plt.rcParams['font.size'] = 10
import plotly.express as px
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.subplots import make_subplots
init_notebook_mode(connected=False)
pio.renderers.default = 'colab'
pio.templates.default = 'ggplot2'

# Data Preprocessing
# ==============================================================================
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

# Background & Problem Statement

ABC is a grocery retailer that needs to improve its sales forecasting method. They want to use machine learning to **consistently meet customer demands** by **maintaining optimal inventory levels** of the products at the right time.

# Sales Dataset Scope

- 33 distinct product types
- 54 stores
- Time Interval: 1 Jan 2013 to 15 Aug 2017

# Objective

Predict sales: for each product type in each store, between 31 Jul 2017 and 15 Aug 2017 (16-day period).

# 1. Data Loading

In [ ]:
prod_df = pd.read_csv('Products_Information.csv', index_col = 'id')

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
prod_df.head()

In [ ]:
# Change 'date' column type to datetime
prod_df['date'] = pd.to_datetime(prod_df['date'])
prod_df.info()

In [ ]:
print(prod_df['date'].min())

In [ ]:
# How many product types & stores?
print('There are', prod_df['product_type'].nunique(), 'types of products and', prod_df['store_nbr'].nunique(), 'stores') # 33 products
print() # 54 stores

# How many data rows do each product have? 90,936
print('Number of sales records in each product is:\n', prod_df.groupby(['product_type']).count())

# How many data rows do each product and each store have? 1,684
print('\nNumber of sales records in combination of store and product is:\n', prod_df.groupby(['product_type', 'store_nbr']).count())

# How many dates?
print('\nThere are', prod_df['date'].nunique(), 'dates. Starting from', prod_df['date'].min(), 'to', prod_df['date'].max())

# Checking for Missing Values

In [ ]:
prod_df.isna().sum()

# No missing values, we are good to proceed

# 2. Exploratory Data Analysis

## 2.1 Splitting Train & Test Sets

In [ ]:
#Splitting for EDA purposes
cutoff_date = pd.to_datetime('2017-07-31')
test_df = prod_df[prod_df['date'] >= cutoff_date]
train_df = prod_df[prod_df['date'] < cutoff_date]

In [ ]:
print('Number of training rows:', train_df.index.nunique())
print('Number of test rows:', test_df.index.nunique())

In [ ]:
train_df.describe()

In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
test_df.head()

## 2.3 Target Variable: Sales

### 2.3.1 Normality Checking

In [ ]:
# Checking normality using box plots: pay attention to symmetry & outliers
plt.figure(figsize=(40,10))
plt.ylim(0,7000)
dep_boxplot = sns.boxplot(x="product_type", y="sales", data=train_df)

Each product's sales differs in terms of range & outlier presence. A lot of them don't have symmetric boxplots and have a lot of outliers, which means their distributions are not normal.

We might need to do log transformations?

### 2.3.2 Total Sales

In [ ]:
# Monthly Average Sales

# Set month
train_time = train_df.set_index('date').resample('M').sales.mean()
train_time = pd.DataFrame(train_time)
train_time['year'] = train_time.index.year


fig = px.line(train_time, x=train_time.index, y='sales', color='year', title='Monthly Average Sales')
fig.update_layout(width=900, height=500)

In [ ]:
# Annual Average Sales
train_time_year = train_df.set_index('date').resample('Y').sales.mean()
train_time_year = pd.DataFrame(train_time_year)
train_time_year['year'] = train_time_year.index.year

fig = px.line(train_time_year, x=train_time_year.index, y='sales', title = 'Annual Average Sales')
fig.update_layout(width=900, height=500)

In [ ]:
# Day of Week Average Sales
# Trying to see if day of week has impact on sales

train_day = train_df.copy()
train_day['year'] = train_df.date.dt.year #add year column
train_day['day of week'] = train_df.date.dt.dayofweek+1 #add day of week column

day_gb = train_day.groupby(['year', 'day of week'])['sales'].mean().reset_index()

In [ ]:
px.line(day_gb, x='day of week', y='sales', color='year', title = 'Day of Week Average Sales')

# Sales is highest on the weekends (day of week > 5)

In [ ]:
# Open Days: Is there any day where the stores close?
train_df_time = train_df[['date', 'sales']].groupby('date').mean()
train_df_time['open'] = 1
full_range = pd.date_range(start=train_df_time.index.min(), end=train_df_time.index.max(), freq='D')
full_df = pd.DataFrame(index=full_range)

full_df = full_df.merge(train_df_time['open'], left_index=True, right_index=True, how='left').fillna(0)

fig = px.line(full_df, x=full_df.index, y='open', title='Open Days')
fig.update_layout(width=700, height=700)

In [ ]:
pd.date_range(start=train_df_time.index.min(), end=train_df_time.index.max(), freq='D').difference(train_df_time.index)

In [ ]:
print('The store holidays are on', pd.date_range(start=train_df_time.index.min(), end=train_df_time.index.max(), freq='D').difference(train_df_time.index).values)

In [ ]:
#checking more holidays (where sales were 0 throughout the years on certain date)

train_df_time = train_df[~((train_df['date']<'2017-04-20') & (train_df['store_nbr']==52))] #excluding store_nbr 52, which just opened on 20 Apr 2017
train_df_time['m-d'] = train_df_time['date'].dt.strftime('%m-%d') #
train_df_time = train_df_time.groupby(['m-d', 'store_nbr'])['sales'].sum().reset_index()

train_df_time[train_df_time['sales']==0]

In [ ]:
#checking store 25 & 36 sales in January 1st
train_df_str2536 = train_df[train_df['store_nbr'].isin([25,36])]
train_df_str2536 = train_df_str2536[(train_df_str2536['date'].dt.month==1) & (train_df_str2536['date'].dt.day==1)].copy()

result = train_df_str2536.groupby(['store_nbr','date'])['sales'].sum().reset_index()

In [ ]:
train_df_str2536.groupby(['store_nbr','date'])['sales'].sum().reset_index()

## 2.4 Independent Variables

There are 4 independent variables provided in the dataset: special_offer, date, store_nbr, and product_type.

store_nbr & product_type are categorical, as well as day of week. We only need to check normality for special_offer.

### 2.4.1 special_offer

In [ ]:
# Draw box plot for special_offer only
plt.figure(figsize=(5,5))
train_df.boxplot(['special_offer'])
plt.title('Special Offer Normality Check')
plt.show()
# A lot of outliers

In [ ]:
#plotting special offer trend
special_offer = train_df.set_index('date').resample('M')['special_offer', 'sales'].mean().reset_index() #grouping by month
fig = px.line(special_offer, x='date', y='special_offer', title='Special Offer Trend')
fig.update_layout(width=2000, height=500)

In [ ]:
#plotting special offer and sales
special_offer = train_df.set_index('date').resample('D')['special_offer', 'sales'].mean().reset_index() #grouping by day
special_offer = special_offer[special_offer['date'] >= pd.to_datetime('2014-05-31')] #date when special offers were initiated

fig = px.scatter(special_offer, x='special_offer', y='sales', trendline='ols', color_discrete_sequence=['dodgerblue'], trendline_color_override='red', title='Special Offer vs Sales Trend')
fig.update_layout(width=2000, height=500)

There is some positive correlation between special_offer and sales. We need to include that in our feature engineering & modeling stage later.

In [ ]:
#special offers performance by stores
special_offer_stores = train_df.groupby(['store_nbr', 'date'])[['sales', 'special_offer']].mean().reset_index()

In [ ]:
fig = make_subplots(rows=18, cols=3, subplot_titles=[f'Store {store}' for store in special_offer_stores.store_nbr.unique()])
n=1
for row in range(1, 19):
    for col in range(1, 4):
        if n > len(special_offer_stores.store_nbr.unique()):
            break
        df = special_offer_stores[special_offer_stores['store_nbr'] == n]
        n += 1

        # Generate scatter plot with trendline for each store
        px_fig = px.scatter(df, x='special_offer', y='sales', trendline='ols',
                            color_discrete_sequence=['dodgerblue'], trendline_color_override='red')

        for trace in px_fig['data']:
            fig.add_trace(trace, row=row, col=col)

fig.update_layout(height=4000, width=2000, title_text = 'Special Offers Trends by Stores')
fig.update_xaxes(title_text='special_offer')
fig.update_yaxes(title_text='sales')

fig.show()

In [ ]:
#special offers performance by products
special_offer_products = train_df.groupby(['product_type', 'date'])[['sales', 'special_offer']].mean().reset_index()

In [ ]:
unique_product_types = special_offer_products.product_type.unique()
fig = make_subplots(rows=11, cols=3, subplot_titles=[f'{product}' for product in special_offer_products.product_type.unique()])
n=0
for row in range(1, 12):
    for col in range(1, 4):
        if n >= len(unique_product_types):  # Ensure n does not exceed the number of unique product types
            break
        product_type = unique_product_types[n]
        df = special_offer_products[special_offer_products['product_type'] == product_type]
        n += 1

        # Generate scatter plot with trendline for each store
        px_fig = px.scatter(df, x='special_offer', y='sales', trendline='ols',
                            color_discrete_sequence=['dodgerblue'], trendline_color_override='red')

        for trace in px_fig['data']:
            fig.add_trace(trace, row=row, col=col)

fig.update_layout(height=4000, width=2000, title_text = 'Special Offers Trends by Products')
fig.update_xaxes(title_text='special_offer')
fig.update_yaxes(title_text='sales')

fig.show()

### 2.3.3 Store Performance

In [ ]:
#store's daily sales performance
train_df['store_nbr'] = train_df['store_nbr'].astype('category')
train_store = train_df.groupby(['date', 'store_nbr']).sales.mean().reset_index()


fig = px.line(train_store, x='date', y='sales', color='store_nbr', title='Daily Sales Average by Store Number')
fig.update_layout(width=1700, height=1500)

In [ ]:
#boxplot
px.box(train_store, x='store_nbr', y='sales', color='store_nbr', title='Average Sales by Store Number')

In [ ]:
store_day = train_df.copy()
store_day['year'] = store_day.date.dt.year
store_day['year'] = store_day['year'] .astype(int)
store_day = store_day.groupby(['store_nbr','date','year'])['sales'].mean().reset_index()

fig = make_subplots(rows=18, cols=3, subplot_titles=[f'Store {store}' for store in store_day.store_nbr.unique()])
n=1
for row in range (1,19):
  for col in range(1,4):
    df = store_day[store_day['store_nbr'] == n]
    n += 1

    px_fig = px.line(df, x='date', y='sales', color='year')

    for trace in px_fig['data']:
      fig.add_trace(trace, row=row, col=col)

fig.update_layout(height=4000, width=1700, title_text = 'Daily Sales Graph by Store', showlegend=False)
fig.update_xaxes(title_text='year')
fig.update_yaxes(title_text='sales')

fig.show()


In [ ]:
#store's sales ranking
train_store = train_df.copy()
train_store['year'] = train_df['date'].dt.year

train_agg = train_store.groupby(['store_nbr', 'year']).sum(numeric_only=True).reset_index()
total_sales_per_store = train_agg.groupby('store_nbr')['sales'].sum()
sorted_stores = total_sales_per_store.sort_values(ascending=False).index

train_agg['store_nbr'] = train_agg['store_nbr'].astype(str)
train_agg['store_nbr'] = pd.Categorical(train_agg['store_nbr'], categories=sorted_stores.astype(str), ordered=True)

In [ ]:
fig = px.bar(train_agg, x='sales', y='store_nbr', color='year', category_orders={'store_nbr': sorted_stores.astype(str)}, title='Sales Ranking')
fig.update_layout(width=1700, height=1500)
fig.update_xaxes(title_text='cummulative sales')

### 2.3.4 Product Type Preference & Sales Pattern

In [ ]:
#product's daily sales performance
train_prod = train_df.groupby(['date', 'product_type']).sales.mean().reset_index()

fig = px.line(train_prod, x='date', y='sales', color='product_type', title='Sales by Product Type')
fig.update_layout(width=1700, height=1500)

In [ ]:
#boxplot
train_prod = train_df.groupby(['date', 'product_type']).sales.mean().reset_index()

fig = px.box(train_prod, x='product_type', y='sales', color='product_type', title='Average Sales by Product Type')
fig.update_layout(width=1700, height=800)

In [ ]:
product_day = train_df.copy()
product_day['year'] = product_day.date.dt.year
product_day['year'] = product_day['year'] .astype(int)
product_day = product_day.groupby(['product_type','date','year'])['sales'].mean().reset_index()

prod_type = product_day.product_type.unique()
fig = make_subplots(rows=11, cols=3, subplot_titles=[f'{product}' for product in prod_type])
n=0
for row in range (1,12):
  for col in range(1,4):
    df = product_day[product_day['product_type'] == prod_type[n]]
    n += 1

    px_fig = px.line(df, x='date', y='sales', color='year')

    for trace in px_fig['data']:
      fig.add_trace(trace, row=row, col=col)

fig.update_layout(height=4000, width=1700, title_text = 'Daily Sales Graph by Product', showlegend=False)
fig.update_xaxes(title_text='year')
fig.update_yaxes(title_text='sales')

fig.show()


In [ ]:
#product's sales ranking
train_prod = train_df.copy()
train_prod['year'] = train_df['date'].dt.year
train_agg = train_store.groupby(['product_type', 'year']).sum(numeric_only=True).reset_index()
total_sales_per_product = train_agg.groupby('product_type')['sales'].sum()
sorted_product = total_sales_per_product.sort_values(ascending=False).index

train_agg['product_type'] = pd.Categorical(train_agg['product_type'], categories=sorted_product, ordered=True)

fig = px.bar(train_agg, x='sales', y='product_type', color='year', category_orders={'product_type': sorted_product.astype(str)}, title='Product Ranking')
fig.update_layout(width=1700, height=1500)
fig.update_xaxes(title_text='cummulative sales')

In [ ]:
#product sold in store
heatmap_data = train_df.pivot_table(index='store_nbr', columns='product_type', aggfunc='sum', values='sales')

fig = px.imshow(heatmap_data, text_auto=True, title='Product Sold in Stores')
fig.update_layout(width=1700, height=800)

#### Conclusion from EDA:
1. Annual sales and day-of-week sales have distinct sales patterns.
2. All of the stores closed on Dec 25th. In addition, Jan 1st was assumed to be holiday since there were no sales throughout the years, with the exception of store 25 and 36 (only open in 2014).
2. Special offer has positive correlation to sales.
3. Some stores, as well as the special_offer promotions, have only started operating in more recent years. This strengthens the argument of using only 2017 data for the modeling stage.
4. Different patterns are evident in both stores and product types.
5. Some products have zero sales records in the data. Thus, we assume they were not sold in that particular stores.

# **Data Preprocessing**

1. Delete Products Not Sold in Stores

In [ ]:
prod_info = prod_df.copy()

In [ ]:
#combination of store-product not sold
product_not_sold = prod_info.groupby(['store_nbr', 'product_type'])['sales'].sum().reset_index()
product_not_sold = product_not_sold[product_not_sold['sales']==0]
product_not_sold.set_index(['store_nbr', 'product_type'], inplace=True)

In [ ]:
prod_info.set_index(['store_nbr', 'product_type'], inplace=True)

In [ ]:
not_zero = ~prod_info.index.isin(product_not_sold.index)

prod_info_filtered = prod_info[not_zero].reset_index()

2. Delete Records of Holiday

In [ ]:
holiday_dates = pd.date_range(start='2013-01-01', end='2023-01-01', freq='YS').union(pd.date_range(start='2013-12-25', end='2022-12-25', freq='Y')).tolist()

# Apply the conditions to filter the DataFrame
prod_info_filtered = prod_info_filtered[~((prod_info_filtered['date'].isin(holiday_dates)) & (prod_info_filtered['store_nbr'] != 25) & # Store 25 is the only open store on January 1st
                        ~((prod_info_filtered['date'].dt.year == 2014) & (prod_info_filtered['store_nbr'] == 36)))]  # On January 1st, Store 36 only open in 2014

3. Adding Feature Day of Week

In [ ]:
prod_info_filtered['day of week'] = prod_info_filtered['date'].dt.dayofweek+1

4. Adding Sales Lag, Rolling Means

In [ ]:
prod_info_sorted = prod_info_filtered.sort_values(by=['store_nbr', 'product_type', 'date'])

prod_info_sorted['sales_lag21'] = prod_info_sorted.groupby(['store_nbr', 'product_type'])['sales'].shift(21) #lag21
prod_info_sorted['sales_lag365'] = prod_info_sorted.groupby(['store_nbr', 'product_type'])['sales'].shift(365) #lag365
prod_info_sorted['rolling_means7'] = prod_info_sorted.groupby(['store_nbr', 'product_type'])['sales'].rolling(window=7).mean().shift(21).values #7 days rolling mean shifted 21 days

5. Encode

In [ ]:
prod_info_encoded = prod_info_sorted.copy()
label_encoder = LabelEncoder()
label_encoder.fit(prod_info_encoded['product_type']) #encoder for product_type

In [ ]:
prod_info_encoded['encoded_product_type'] = label_encoder.transform(prod_info_encoded['product_type'])

In [ ]:
prod_info_encoded = prod_info_encoded.sort_values(by=['date','store_nbr', 'product_type']).reset_index(drop=True)

6. Splitting Dataset

In [ ]:
cutoff_date1 = pd.to_datetime('2017-07-31')
cutoff_date2 = pd.to_datetime('2017-01-01')
test_df = prod_info_encoded[prod_info_encoded['date'] >= cutoff_date1]
train_df = prod_info_encoded[(prod_info_encoded['date'] >= cutoff_date2) & (prod_info_encoded['date'] < cutoff_date1)]

7. Export to CSV